In [1]:
import os

from dotenv import load_dotenv, find_dotenv
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.question_answering import load_qa_chain 
from langchain.chains import LLMChain
from langchain.schema.messages import HumanMessage
from langchain.callbacks import get_openai_callback
import pandas as pd

from utils.file_handler import Reader, Writer

In [2]:
# check API
load_dotenv()

True

# Limited articles approach(no RAG)

In [3]:
# queries
query_name = "query1"
query = Reader.read_txt(f'query/{query_name}.txt')

In [4]:
# GLOVAL LLM VALUES
MODEL_NAME = 'gpt-4o'
TEMPERATURE = 0.0

# Define llm
llm = ChatOpenAI(model=MODEL_NAME, temperature=TEMPERATURE)

C:\Users\Solo\AppData\Local\Temp\ipykernel_5928\55862162.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model=MODEL_NAME, temperature=TEMPERATURE)


In [5]:
# Ppepare templates
STANDART_TEMPLATE = """
Ти викладач юридичного права України.
Ти намагаєшся відповідати на питання студентів наступним шаблоном:
"[Імʼя особи]-[пункт, частина, статаття, що передбачає покарання за вчинене кримінальне правопорушення, назва статті]."
Статей може бути декілька.]"
Ти використовуєш кримінальний кодекс України.
Якщо відповіді ти не знаєш, відповідай "Не певен, але вважаю що" і намагайся здогадатися, якщо знаєш відповідай "Відповідь така".
{query}
"""
LIMITED_ARTICLES_TEMPLATE = """
Ти спеціаліст по праву. 
Якщо відповіді ти не знаєш, відповідай "Не певен, але вважаю що" і намагайся здогадатися, якщо знаєш відповідай "Відповідь така" 
{query} Якими статтями можна описати діяння учасників?
Стаття 124. Умисне заподіяння тяжких тілесних ушкоджень у разі перевищення меж 
Стаття 125. Умисне легке тілесне ушкодження
Стаття 126. Побої і мордування
Стаття 126-1. Домашнє насильство
Стаття 292. Пошкодження об'єктів магістральних або промислових нафто-, газо-, конденсатопроводів та нафтопродуктопроводів
Стаття 293. Групове порушення громадського порядку
Стаття 300. Ввезення, виготовлення або розповсюдження творів, що пропагують культ 
Стаття 322. Незаконна організація або утримання місць для вживання одурманюючих засобів
Стаття 323. Спонукання неповнолітніх до застосування допінгу
Стаття 324. Схиляння неповнолітніх до вживання одурманюючих засобів
"""

In [6]:
# Define prompt
prompt_template = PromptTemplate(input_variables=['query'], 
                                 template=LIMITED_ARTICLES_TEMPLATE)

print(prompt_template.format(query=query))

# Count the query tokens
with get_openai_callback() as cb:
    result = llm([HumanMessage(content=prompt_template.format(query=query))])
    print(f"\n Total Tokens: {cb.total_tokens}")
    print(f" Prompt Tokens: {cb.prompt_tokens}")
    print(f" Completion Tokens: {cb.completion_tokens}")
    print(f" Total Cost (USD): ${round(cb.total_cost, 3)}")



Ти спеціаліст по праву. 
Якщо відповіді ти не знаєш, відповідай "Не певен, але вважаю що" і намагайся здогадатися, якщо знаєш відповідай "Відповідь така" 
Контекст: Вахтанг, керівник однієї з політичних партій в Україні, перебуваючи у відрядженні у Республіці Польща, дізнався, що його громадянство України було припинено з підстав неналежного оформленні документів під час одержання такого громадянства. Бажаючи продовжувати свою політичну діяльність, Вахтанг домовився із Василем, Петром та Іваном, що ті організують «прихильників», які створять «масовку» під час переходу ним державного кордону України зі сторони Республіки Польща.
11 вересня 2021 року Вахтанг прибув потягом до одного з пунктів пропуску на польсько-українському кордоні, зійшов з нього та у оточенні своїх прихильників (в кількості близько 20 осіб) яких покликали Василь та Петро, почав рухатися до пішохідного переходу. Іван та близько 200 прихильників Вахтанга чекали його на території України, викрикуючи політичні гасла. По

C:\Users\Solo\AppData\Local\Temp\ipykernel_5928\3689684043.py:9: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = llm([HumanMessage(content=prompt_template.format(query=query))])



 Total Tokens: 1130
 Prompt Tokens: 865
 Completion Tokens: 265
 Total Cost (USD): $0.005


In [ ]:
chain = prompt_template | llm 

In [8]:
response = chain.invoke({"query": query})

In [12]:
report = f"The answer was generated by {MODEL_NAME}, temperature={TEMPERATURE}"

In [ ]:
prompt_text = prompt_template.format(query=query)
answer_text = getattr(response, "content", str(response))
text = f"Prompt:\n{prompt_text}\n{report}\nAnswer:\n{answer_text}"
Writer.write_txt(text, query_name)